In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dino_functions import Dinov2

In [ ]:
'''
    Camera Serial to number 
    130322273305 -> 1
    128422270081 -> 2
    127122270512 -> 3
'''

serial_to_number = {
    130322273305: 1,
    128422270081: 2,
    127122270512: 3
}

number_to_serial = {
    1: 130322273305,
    2: 128422270081,
    3: 127122270512
}


In [ ]:
serial_list = [130322273305, 128422270081, 127122270512]



NVIDIA GeForce RTX 2060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 2060', major=7, minor=5, total_memory=5919MB, multi_processor_count=30, uuid=3966ec4e-46d8-062a-bfcc-c186316a2a4a, L2_cache_size=3MB)


In [ ]:
dino = Dinov2()
    demo_image = cv2.cvtColor(demo_image, cv2.COLOR_BGR2RGB)
    demo_image_tensor, demo_image_grid = dino.prepare_image(demo_image)
    demo_image_features = dino.extract_features(demo_image_tensor)
    point_indices = {}
    contact_point_index = {}
    directional_point_index = {}

    for key in sampled_points_2d:
        point_indices[key] = []
        for point in sampled_points_2d[key]:
            col, row = point
            point_indices[key].append(dino.pixel_to_idx([row, col], demo_image_grid, dino.patch_size))
        contact_point_index[key] = dino.pixel_to_idx([contact_point_2d[key][1], contact_point_2d[key][0]], demo_image_grid, dino.patch_size)
        directional_point_index[key] = dino.pixel_to_idx([directional_point_2d[key][1], directional_point_2d[key][0]], demo_image_grid, dino.patch_size)


    serial_numbers = [130322273305, 128422270081, 127122270512]
    best_serial = None
    best_distance_map = None
    best_inference_contact_point = None
    best_inference_directional_point = None
    best_distance = float('inf')

    for serial in serial_numbers:
        inference_color_image = np.load("resources/" + object_name + "/inference_color_image_" + str(serial) + ".npy")
        inference_color_image = cv2.cvtColor(inference_color_image, cv2.COLOR_BGR2RGB)

        inference_color_image_tensor, inference_color_image_grid = dino.prepare_image(inference_color_image)
        inference_color_image_features = dino.extract_features(inference_color_image_tensor)
        print("extracted features for serial: ", serial)

        total_distance = 0
        distance_map = {}
        inference_contact_point = {}
        inference_directional_point = {}

        for key in point_indices:
            distance_map[key] = None
            raw_distance = None
            for i, point_index in enumerate(point_indices[key]):
                raw_distance = dino.compute_feature_distance(point_index, demo_image_features, inference_color_image_features)
                raw_distance = np.reshape(raw_distance, (inference_color_image_grid[0], inference_color_image_grid[1]))
                raw_distance = cv2.resize(raw_distance, (inference_color_image_tensor.shape[2], 
                                                inference_color_image_tensor.shape[1]), interpolation=cv2.INTER_CUBIC)
                threshold = np.percentile(raw_distance, 0.005)
                distance = raw_distance < threshold
                
                if distance_map[key] is None:
                    distance_map[key] = distance
                else:
                    distance_map[key] = np.logical_or(distance_map[key], distance)

            points = np.argwhere(distance_map[key])
            for point in points:
                point_coords = np.round(point).astype(int)
                point_distance = raw_distance[point_coords[0], point_coords[1]]
                # print("Point Distance: ", point_distance)
                total_distance += point_distance
                
            raw_contact_point_distance = dino.compute_feature_distance(contact_point_index[key], demo_image_features, inference_color_image_features)
            raw_contact_point_distance = np.reshape(raw_contact_point_distance, (inference_color_image_grid[0], inference_color_image_grid[1]))
            raw_contact_point_distance = cv2.resize(raw_contact_point_distance, (inference_color_image_tensor.shape[2],
                                                    inference_color_image_tensor.shape[1]), interpolation=cv2.INTER_CUBIC)
            threshold = np.percentile(raw_contact_point_distance, 0.005)
            contact_point_distance = raw_contact_point_distance < threshold   
            

            raw_directional_point_distance = dino.compute_feature_distance(directional_point_index[key], demo_image_features, inference_color_image_features)
            raw_directional_point_distance = np.reshape(raw_directional_point_distance, (inference_color_image_grid[0], inference_color_image_grid[1]))
            raw_directional_point_distance = cv2.resize(raw_directional_point_distance, (inference_color_image_tensor.shape[2],
                                                        inference_color_image_tensor.shape[1]), interpolation=cv2.INTER_CUBIC)
            threshold = np.percentile(raw_directional_point_distance, 0.005)
            directional_point_distance = raw_directional_point_distance < threshold
            
            inference_contact_point[key] = np.mean(np.argwhere(contact_point_distance), axis=0)
            inference_directional_point[key] = np.mean(np.argwhere(directional_point_distance), axis=0)

            cp_coords = np.round(inference_contact_point[key]).astype(int)
            cp_distance_value = raw_contact_point_distance[cp_coords[0], cp_coords[1]]

            dp_coords = np.round(inference_directional_point[key]).astype(int)
            dp_distance_value = raw_directional_point_distance[dp_coords[0], dp_coords[1]]

            total_distance += cp_distance_value + dp_distance_value

            # total_distance += np.sum(contact_point_distance) + np.sum(directional_point_distance)
        print("total distance for serial: ", serial, " is: ", total_distance)
        
        if total_distance < best_distance:
            best_distance = total_distance
            best_serial = serial
            best_distance_map = distance_map
            best_inference_contact_point = inference_contact_point
            best_inference_directional_point = inference_directional_point

    #show best camera inference image
    print("Best Serial: ", best_serial)
    best_color_image = np.load("resources/" + object_name + "/inference_color_image_" + str(best_serial) + ".npy")
    # best_color_image = cv2.cvtColor(best_color_image, cv2.COLOR_BGR2RGB)
    display_image(best_color_image, "Best Inference Image")

    distance_map = best_distance_map
    inference_contact_point = best_inference_contact_point
    inference_directional_point = best_inference_directional_point
        

In [ ]:
 dino = Dinov2()
    demo_image = cv2.cvtColor(demo_image, cv2.COLOR_BGR2RGB)
    
    demo_image_tensor, demo_image_grid = dino.prepare_image(demo_image)
    demo_image_features = dino.extract_features(demo_image_tensor)

    indices_sampled_points_2d = {}
    demo_contact_point_index = {}
    demo_directional_point_index = {}
    for key in sampled_points_2d:
        indices_sampled_points_2d[key] = []
        for point in sampled_points_2d[key]:
            col, row = point
            indices_sampled_points_2d[key].append(dino.pixel_to_idx([row, col], demo_image_grid, dino.patch_size))
        demo_contact_point_index[key] = dino.pixel_to_idx([contact_point_2d[key][1], contact_point_2d[key][0]], demo_image_grid, dino.patch_size)
        demo_directional_point_index[key] = dino.pixel_to_idx([directional_point_2d[key][1], directional_point_2d[key][0]], demo_image_grid, dino.patch_size)

    
    camera_serials = [130322273305, 128422270081, 127122270512]
    best_serial = None
    min_distance = float('inf')
    best_inference_contact_point = None
    best_inference_directional_point = None
    best_distance_map = None

    for serial in camera_serials:
        inference_color_image = np.load("resources/" + object_name + "/inference_color_image_" + str(serial) + ".npy")
        inference_color_image = cv2.cvtColor(inference_color_image, cv2.COLOR_BGR2RGB)

        inference_image_tensor, inference_image_grid = dino.prepare_image(inference_color_image)
        inference_image_features = dino.extract_features(inference_image_tensor)

        total_distance = 0
        dist_map = {}
        contact_pt = {}
        directional_pt = {}

        for key in indices_sampled_points_2d:
            dist_map[key] = None
            for i, point_index in enumerate(indices_sampled_points_2d[key]):
                distance = dino.compute_feature_distance(point_index,
                                                         demo_image_features,
                                                         inference_image_features)
                distance = np.reshape(distance, (inference_image_grid[0], inference_image_grid[1]))
                distance = cv2.resize(distance, (inference_image_tensor.shape[2],
                                                 inference_image_tensor.shape[1]),
                                                 interpolation=cv2.INTER_CUBIC)
                threshold = np.percentile(distance, 0.005)
                distance_mask = distance < threshold
                distance_mask = distance_mask.astype(np.uint8)
                distance *= distance_mask

                if dist_map[key] is None:
                    dist_map[key] = distance
                else:
                    dist_map[key] = np.logical_or(dist_map[key], distance>0)

                total_distance += np.sum(distance)

            contact_pt_distance = dino.compute_feature_distance(demo_contact_point_index[key],
                                                                demo_image_features,
                                                                inference_image_features)
            contact_pt_distance = np.reshape(contact_pt_distance, (inference_image_grid[0], inference_image_grid[1]))
            contact_pt_distance = cv2.resize(contact_pt_distance, (inference_image_tensor.shape[2],
                                                                 inference_image_tensor.shape[1]),
                                                                 interpolation=cv2.INTER_CUBIC)
            contact_pt_threshold = np.percentile(contact_pt_distance, 0.005)
            contact_pt_distance_mask = contact_pt_distance < contact_pt_threshold
            contact_pt_distance_mask = contact_pt_distance_mask.astype(np.uint8)
            contact_pt_distance *= contact_pt_distance_mask
            contact_pt[key] = np.mean(np.argwhere(contact_pt_distance_mask>0), axis=0)

            directional_pt_distance = dino.compute_feature_distance(demo_directional_point_index[key],
                                                                   demo_image_features,
                                                                   inference_image_features)
            directional_pt_distance = np.reshape(directional_pt_distance, (inference_image_grid[0], inference_image_grid[1]))
            directional_pt_distance = cv2.resize(directional_pt_distance, (inference_image_tensor.shape[2],
                                                                       inference_image_tensor.shape[1]),
                                                                       interpolation=cv2.INTER_CUBIC)
            directional_pt_threshold = np.percentile(directional_pt_distance, 0.005)
            directional_pt_distance_mask = directional_pt_distance < directional_pt_threshold
            directional_pt_distance_mask = directional_pt_distance_mask.astype(np.uint8)
            directional_pt_distance *= directional_pt_distance_mask 
            directional_pt[key] = np.mean(np.argwhere(directional_pt_distance_mask>0), axis=0)

            total_distance += np.sum(contact_pt_distance)
            total_distance += np.sum(directional_pt_distance)